In [ ]:
#!/usr/bin/env python3
"""
RTSE Data Simulator with per-bulk folders, timestamps, and rounded metadata.

Generates NCS CSV + JSON for:
  └─ output_root/
       └─ <BulkName>/
           └─ <CaseName>/
               ├─ <Bulk>_<Case>_0000_<timestamp>.csv
               └─ <Bulk>_<Case>_0000_<timestamp>.json
"""

import os
import glob
import json
import random
import pandas as pd
import numpy as np
from datetime import datetime
from tqdm import tqdm

# ───────────────────────────────────────────────────────────────────────────────
# 1) USER CONFIGURATION
# ───────────────────────────────────────────────────────────────────────────────

optical_props_dir    = r"C:\Users\mlell\OneDrive\Desktop\OpProps-Nanda"
output_root          = r"C:\Users\mlell\OneDrive\Desktop\Newdata"
sims_per_case        = 30000   # files per case

bulk_thickness_range = (10.0, 500.0)  # nm
ema_thickness_range  = (2.0, 100.0)   # nm
void_fraction_range  = (0.1, 0.9)     # unitless
oxide_thickness      = 1.75           # nm
AOI_degrees          = 70.0           # incidence angle

# ───────────────────────────────────────────────────────────────────────────────
# 2) CORE FUNCTIONS (as before)
# ───────────────────────────────────────────────────────────────────────────────

def Bruggeman_EMA_Roussel(M1, M2, c):
    wv = M1['Wavelength (nm)'].to_numpy()
    N1, N2 = M1['N'].to_numpy(), M2['N'].to_numpy()
    p = N1 / N2
    b = 0.25 * ((3*c - 1) * ((1/p) - p) + p)
    z = b + np.sqrt(b*b + 0.5)
    e = z * N1 * N2
    e1, e2 = e.real, e.imag
    mag = np.sqrt(e1*e1 + e2*e2)
    n = np.sqrt((mag + e1)/2)
    k = np.sqrt((mag - e1)/2)
    df = pd.DataFrame({'Wavelength (nm)': wv, 'N': n + 1j*k})
    df.name = f"EMA_{M1.name}_{M2.name}_{c:.2f}"
    return df

def Snells_Law(Structure, AOI):
    Nmat = np.stack([df["N"].to_numpy() for df in Structure])
    L, P = Nmat.shape
    angles = np.zeros((L,P), dtype=complex)
    angles[0] = np.radians(AOI)
    for i in range(1, L):
        angles[i] = np.arcsin((Nmat[i-1]/Nmat[i]) * np.sin(angles[i-1]))
    return angles

def fresnel_coefficients(N, angles):
    n1, n2 = N[:-1], N[1:]
    t1, t2 = angles[:-1], angles[1:]
    cos1, cos2 = np.cos(t1), np.cos(t2)
    ds = n1*cos1 + n2*cos2
    dp = n2*cos1 + n1*cos2
    rs = (n1*cos1 - n2*cos2)/ds
    ts = (2*n1*cos1)/ds
    rp = (n2*cos1 - n1*cos2)/dp
    tp = (2*n1*cos1)/dp
    return rs, rp, ts, tp

def Scattering_Matrix(N, angles, d, lam, r, t):
    L, P = N.shape
    d = d[:,None]; lam=lam[None,:]
    E = (2*np.pi/lam)*N[1:-1]*d*np.cos(angles[1:-1])
    prop = np.zeros((L-2,P,2,2), dtype=complex)
    prop[:,:,0,0] = np.exp(-1j*E)
    prop[:,:,1,1] = np.exp( 1j*E)
    interf = np.zeros((L-1,P,2,2), dtype=complex)
    interf[:,:,0,0] = 1/t; interf[:,:,0,1] = r/t
    interf[:,:,1,0] = r/t; interf[:,:,1,1] = 1/t
    S = interf[0]
    for i in range(1, L-1):
        S = S @ prop[i-1] @ interf[i]
    return S

def SE_Sim(Structure, AOI, d, write_data=False, NCS=True):
    wv = Structure[0]['Wavelength (nm)'].to_numpy()
    Nmat = np.stack([df["N"].to_numpy() for df in Structure])
    angles = Snells_Law(Structure, AOI)
    rs, rp, ts, tp = fresnel_coefficients(Nmat, angles)
    Ss = Scattering_Matrix(Nmat, angles, d, wv, rs, ts)
    Sp = Scattering_Matrix(Nmat, angles, d, wv, rp, tp)
    Rp = Sp[:,1,0]/Sp[:,0,0]
    Rs = Ss[:,1,0]/Ss[:,0,0]
    rho = np.conj(Rp/Rs)
    psi = np.arctan(np.abs(rho)).real
    delta = np.unwrap(np.angle(rho))
    Nval = np.cos(2*psi).real
    C = (np.sin(2*psi)*np.cos(delta)).real
    S = (np.sin(2*psi)*np.sin(delta)).real
    if NCS:
        return pd.DataFrame({'Wavelength (nm)': wv, 'N': Nval, 'C': C, 'S': S})
    else:
        return pd.DataFrame({
            'Wavelength (nm)': wv,
            'Psi':   psi*180/np.pi,
            'Delta': delta*180/np.pi
        })

# ───────────────────────────────────────────────────────────────────────────────
# 3) LOAD MATERIALS
# ───────────────────────────────────────────────────────────────────────────────

def load_materials(opt_dir):
    mats = {}
    for fp in glob.glob(os.path.join(opt_dir, "*.csv")):
        df = pd.read_csv(fp)
        name = os.path.splitext(os.path.basename(fp))[0]
        df.name = name
        if {'n','k'}.issubset(df.columns):
            df['N'] = df['n'] + 1j*df['k']
        mats[name] = df
    return mats

materials = load_materials(optical_props_dir)

# fix key if your CSV is named "CdTe-OpProp.csv"
if "CdTe-OpProp" in materials:
    materials["CdTe"] = materials.pop("CdTe-OpProp")

Void      = materials["Void"]
Oxide     = materials["NTVE_JAW"]
Substrate = materials["Si_JAW"]

bulk_materials = {
    "CdTe":   materials["CdTe"],
    "a-Si":   materials["a-Si"],
    "Sb2Se3": materials["Sb2Se3"],
}

# ───────────────────────────────────────────────────────────────────────────────
# 4) STRUCTURAL CASE CONFIG
# ───────────────────────────────────────────────────────────────────────────────

case_configs = {
    "Case1_NucOnly": {
        "surfEMA": False, "nucEMA": True
    },
    "Case2_NoEMA": {
        "surfEMA": False, "nucEMA": False
    },
    "Case3_SurfOnly": {
        "surfEMA": True,  "nucEMA": False
    },
    "Case4_BothEMAs": {
        "surfEMA": True,  "nucEMA": True
    },
}

# ───────────────────────────────────────────────────────────────────────────────
# 5) DRIVER: per-bulk → per-case → sims_per_case
# ───────────────────────────────────────────────────────────────────────────────

for bulk_name, bulk_df in bulk_materials.items():
    for case_name, cfg in case_configs.items():
        out_dir = os.path.join(output_root, bulk_name, case_name)
        os.makedirs(out_dir, exist_ok=True)

        desc = f"{bulk_name}-{case_name}"
        for i in tqdm(range(sims_per_case), desc=desc):
            # random params
            t_bulk = random.uniform(*bulk_thickness_range)
            c1      = random.uniform(*void_fraction_range)
            c2      = random.uniform(*void_fraction_range)
            t1      = random.uniform(*ema_thickness_range) if cfg["surfEMA"] else None
            t2      = random.uniform(*ema_thickness_range) if cfg["nucEMA"]  else None

            # build layers list
            layers = [Void]
            if cfg["surfEMA"]:
                surf = Bruggeman_EMA_Roussel(bulk_df, Void, c1)
                surf.name = f"SurfEMA_c{c1:.2f}"
                layers.append(surf)
            layers.append(bulk_df)
            if cfg["nucEMA"]:
                nuc = Bruggeman_EMA_Roussel(bulk_df, Void, c2)
                nuc.name = f"NucEMA_c{c2:.2f}"
                layers.append(nuc)
            layers.extend([Oxide, Substrate])

            # thickness array (skip ambient & substrate)
            d_list = []
            if cfg["surfEMA"]: d_list.append(t1)
            d_list.append(t_bulk)
            if cfg["nucEMA"]:  d_list.append(t2)
            d_list.append(oxide_thickness)
            d_arr = np.array(d_list)

            # simulate
            df = SE_Sim(layers, AOI_degrees, d_arr, write_data=False, NCS=True)

            # timestamp
            ts = datetime.now().strftime("%Y%m%d-%H%M%S-%f")

            # filenames
            fname_base = f"{bulk_name}_{case_name}_{i:04d}_{ts}"
            csv_fp     = os.path.join(out_dir, fname_base + ".csv")
            json_fp    = os.path.join(out_dir, fname_base + ".json")

            # write CSV
            df.to_csv(csv_fp, index=False)

            # prepare & write JSON (rounded to 2 decimals)
            meta = {
                "case":    case_name,
                "bulk":    bulk_name,
                "AOI_deg": round(AOI_degrees, 2),
                "thickness": {
                    **({"SurfaceEMA": round(t1, 2)} if t1 is not None else {}),
                    "Bulk":       round(t_bulk, 2),
                    **({"NucEMA":   round(t2, 2)} if t2 is not None else {}),
                    "Oxide":      round(oxide_thickness, 2)
                },
                "void_fraction": {
                    **({"SurfaceEMA": round(c1, 2)} if cfg["surfEMA"] else {}),
                    **({"NucEMA":     round(c2, 2)} if cfg["nucEMA"]  else {})
                },
                "layers": [L.name for L in layers]
            }
            with open(json_fp, "w") as fj:
                json.dump(meta, fj, indent=2)